### Setup e Variáveis

In [ ]:
from core.bemol_validator import BemolValidator
from notebooks.core.bemol_lakestorage import BemolLakeStorage
from core.bemol_controller import BemolController
from core.bemol_logger import BemolLogger
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat_ws

In [ ]:
# Instanciando classe BemolLogger
logger = BemolLogger("silver_users")

# Configurando Spark com Delta Lake
spark = (
  SparkSession.builder
  .appName("TransformacaoSilverUsuarios")
  .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0")
  .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
  .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
  .getOrCreate()
)

# Instanciando classe de leitura/escrita de dados BemolLakeStorage
lakehouse = BemolLakeStorage(spark, logger)

In [ ]:
# Definindo paths de origem e destino
origin_path = "../data/bronze/users/"
destination_path = "../data/silver/users/"

# Path para salvar os dados de monitoramento
destination_path_monitor = "../data/monitoring/"

### Leitura

In [ ]:
# Lendo dados da camada bronze através do método read_bronze
df_users = lakehouse.read_bronze(origin_path)

### Tranformações

In [ ]:
# Remove duplicatas
df_users = df_users.dropDuplicates(["id"])

# Adiciona a coluna full_name
df_users_silver = df_users.withColumn("full_name", concat_ws(" ", col("first_name"), col("last_name")))

# Remove colunas indesejadas
df_users_silver = df_users_silver.select("id", "username", "full_name", "email", "phone", "city")

# Valida a coluna de e-mail com a classe BemolValidator
df_users_silver = BemolValidator.validate_email(df_users_silver, "email")

### Escrita

In [ ]:
# Adiciona coluna de controle através do método control_field
df_users_silver = BemolController.control_field(df_users_silver, layer="silver")

In [ ]:
# Grava os dados na camada Silver no formato Delta, utilizando o modo overwrite como padrão através do método write_silver
lakehouse.write_silver(df_users_silver, destination_path, table_name="silver_users")

In [ ]:
# Grava os dados de monitoramento no formato Delta, utilizando o modo overwrite como padrão através do método export_delta
lakehouse.monitor.export_delta(spark, destination_path_monitor)